In [ ]:
import datasets
from src import data_loading

dataset = datasets.load_dataset("lmarena-ai/arena-human-preference-140k")
data = data_loading.load_training_data(dataset["train"].to_pandas())
print(f"Successfully loaded {len(data.entries)} entries")

In [ ]:
from src.utils import data_split
from src.models.dense_network_model import DenseNetworkModel

training_data, val_data = data_split.train_val_split(data, val_fraction=0.2, seed=42)

model = DenseNetworkModel(
    hidden_dims=[512, 512, 256, 256, 128, 128, 64, 64],
    model_id_embedding_dim=64,
    learning_rate=5e-6
)

model.train(training_data, val_data, epochs=3_500, batch_size=512)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

history = model.get_history()

def plot_loss(axes: plt.Axes, values: np.ndarray, title: str):
    t = np.arange(len(values))
    log_loss = np.log(values)
    sliding_mean = np.convolve(log_loss, np.ones(10)/10, mode='valid')
    axes.plot(t, log_loss, label="log(loss)")
    axes.plot(t[9:], sliding_mean, label="Sliding mean (10)")

    min_sliding_mean = np.min(sliding_mean)
    axes.axhline(y=min_sliding_mean, color='r', linestyle='--', label=f'Min: {min_sliding_mean:.3f}')

    axes.legend()
    
_, axes = plt.subplots(2, 1, figsize=(12, 8))

plot_loss(axes[0], history.total_loss, "Training loss")
plot_loss(axes[1], history.val_loss, "Validation loss")

plt.show()

In [ ]:
from src.data_models.data_models import InputData

example_prompts = [e.user_prompt for e in val_data.entries]
results = model.predict(InputData(
        prompts=example_prompts,
        model_names=model._model_encoder.names,
    ),
    batch_size=512,
).scores

sorted_results = sorted(
    [(name, float(np.mean(scores)), float(np.std(scores))) for name, scores in results.items()],
    key=lambda x: x[1],
    reverse=True
)

for name, mean_score, std_score in sorted_results:
    print(f"{(name+":"):<{max(len(name) for name, _, _ in sorted_results)+1}s} {mean_score:.6f} ± {std_score:.6f}")